Import packages and processed run metrics

In [47]:
# packages
import re
import numpy as np
import pandas as pd

# control
project_names = [
    #'femnist--s02',
    'sst2',
    'pubmed',
    'cifar--alpha5.0',
    'cifar--alpha0.5',
    'cifar--alpha0.1'
]
validation_metric = 'Results/val_acc'

# read each data set
df = []
print('input datasets:')
for project_name in project_names:
    temp = pd.read_csv(f'decay--{project_name}.csv')
    print('\t', temp.shape)
    df.append(temp)

df = pd.concat(df, axis=0, ignore_index=True)
print('all runs:', df.shape)

# remove any values from list if still inside one
# convert to numeric
metrics_to_reformat = [
    name for name, type in zip(df.columns, df.dtypes)
    if re.search('Results', name)
        and type == 'object'
]
df[metrics_to_reformat] = df[metrics_to_reformat].applymap(lambda x: re.sub('[\[\]]', '', x) if isinstance(x, str) else x).astype('float')

input datasets:
	 (1200, 35)
	 (780, 28)
	 (520, 27)
	 (520, 27)
	 (520, 27)
all runs: (3540, 36)


Write function to return summaries of run's metric performance

In [48]:
def process_run_metrics(filtered_df, method='exact'):

    ## Process metrics and get top runs for each
    # are large or small metric values are desirable?
    descending_metrics = [name for name in filtered_df.columns if re.search('Results.*test', name)]
    ascending_metrics = [
        descending_metrics.pop(descending_metrics.index(name))
        for name in descending_metrics
        if re.search('std', name)
    ]
    metrics = descending_metrics + ascending_metrics

    # all non-metrics columns are used to identify the experimental run
    filtered_runs = filtered_df[[
        name for name in filtered_df.columns
        if name not in metrics
    ]]

    # rank the metrics
    ranked_descending = filtered_df.groupby(['dataset'])[descending_metrics].rank(
        method='min',
        ascending=False
    )
    ranked_ascending = filtered_df.groupby(['dataset'])[ascending_metrics].rank(
        method='min',
        ascending=True
    )

    # combine and sort the ranked_metrics
    ranked_metrics = pd.concat([ranked_descending, ranked_ascending], axis=1)
    ranked_metrics = ranked_metrics[sorted(ranked_metrics.columns)]
    filtered_ranks = filtered_runs.join(ranked_metrics)


    ## Return table of top ranks for each dataset
    filtered_top_ranks = filtered_ranks.copy()
    filtered_top_ranks[metrics] = filtered_top_ranks[metrics].applymap(lambda x: x if x <= 3 else pd.NA)


    ## Manipulate rank data to be summarized by runs and metrics
    # convert to long format
    # filter to top ranks
    long_filtered_ranks = pd.melt(filtered_ranks, id_vars=filtered_runs.columns, var_name='metric')
    top_filtered_metrics = long_filtered_ranks.loc[long_filtered_ranks.value <= 3].copy()  # top 3 runs

    # compute rank summaries to understand what runs are top overall
    top_filtered_metrics['rank_one_ind'] = (top_filtered_metrics.value == 1)
    top_filtered_metrics['rank_two_ind'] = (top_filtered_metrics.value == 2)
    top_filtered_metrics['rank_three_ind'] = (top_filtered_metrics.value == 3)
    top_filtered_metrics.replace(False, pd.NA, inplace=True)

    # summarized metric ranks for run type
    rank_summary_columns = ['rank_one_ind', 'rank_two_ind', 'rank_three_ind', 'value']
    id_columns = ['method', 'finetune']
    run_summary = top_filtered_metrics.groupby(id_columns)[rank_summary_columns].count()

    # summarize metric ranks for metric choice
    metric_summary = top_filtered_metrics.loc[top_filtered_metrics.method == method]
    metric_summary = metric_summary.sort_values(by='metric').groupby('metric')[rank_summary_columns].count()


    return (
        run_summary,
        metric_summary,
        filtered_top_ranks
    )


Filter data to runs of interest
Apply summary function to filtered dataset

In [49]:
subset_df = df
print('input dataset:', df.shape)

## row (run) filtering
# remove extra hyper-parameter searches
subset_df = subset_df.loc[(subset_df.n_epochs < 6)]
print('\t reduce to 3 or fewer local update steps:', subset_df.shape)
subset_df = subset_df.loc[(subset_df.K < 6) | (subset_df.method != 'pfedme')]
print('\t reduce to 3 or fewer local meta-learning steps for pfedme:', subset_df.shape)
subset_df = subset_df.loc[((10 * subset_df.beta).astype('Int64') % 2 == 0) | (subset_df.method != 'exact')]
print('\t reduce beta grid for decay:', subset_df.shape)

## column (metric) filtering
subset_df = subset_df[[
    name for name in subset_df.columns
    if not re.search('f1', name)
       and not re.search('loss', name)
]]

input dataset: (3540, 36)
	 reduce to 3 or fewer local update steps: (2770, 36)
	 reduce to 3 or fewer local meta-learning steps for pfedme: (2690, 36)
	 reduce beta grid for decay: (2490, 36)


In [50]:
## get best runs for each group
# regardless of finetuning
best_runs = subset_df.loc[
    subset_df.groupby(['dataset', 'method', 'finetune']) \
    [validation_metric].idxmax()  # return index of max validation metric
]

# treat finetuning groups as seperate
ft_yes = best_runs.loc[best_runs.finetune == 1]
ft_no = best_runs.loc[best_runs.finetune == 0]

# show best runs
best_runs.head()

,Unnamed: 0,method,finetune,dataset,n_epochs,batch_size,lr,beta,regular_weight,K,Results/test_acc,Results/test_acc_bottom_decile,Results/test_acc_std,Results/val_acc,Results_unseen/test_acc,Results_unseen/test_acc_bottom_decile,Results_unseen/test_acc_std,Results_weighted/test_acc,Results_weighted_unseen/test_acc,alpha
3358,338,ditto,0,cifar--alpha0.1,3,NaN,0.50,NaN,0.50,NaN,0.510895,0.282051,0.176661,0.487736,0.068658,0.000000,0.104141,0.490770,0.048854,0.1
3405,385,ditto,1,cifar--alpha0.1,1,NaN,0.01,NaN,0.05,NaN,0.410931,0.178082,0.203197,0.412543,0.095329,0.000000,0.181180,0.407696,0.088058,0.1
3452,432,exact,0,cifar--alpha0.1,3,NaN,0.05,0.8,NaN,NaN,0.587413,0.390244,0.146089,0.554641,0.633594,0.492958,0.128820,0.562935,0.609771,0.1
3507,487,exact,1,cifar--alpha0.1,3,NaN,0.10,0.2,NaN,NaN,0.571335,0.379487,0.154435,0.566214,0.554705,0.403670,0.117521,0.554304,0.599517,0.1
3332,312,fedavg,0,cifar--alpha0.1,3,NaN,0.05,NaN,NaN,NaN,0.591100,0.402174,0.144218,0.553132,0.629551,0.500000,0.125442,0.564013,0.606755,0.1


For all best runs, regardless of finetuning, produce summaries

In [51]:
(all_run_summary, all_metric_summary, temp) = process_run_metrics(best_runs)
print('all runs:', all_run_summary.shape)
print('all metrics:', all_metric_summary.shape)

# run summary
all_run_summary.sort_values(by='value', ascending=False)

all runs: (12, 4)
all metrics: (8, 4)


rank_one_ind  rank_two_ind  rank_three_ind  value
method finetune                                                   
exact  0                   11             7               5     23
       1                    7             4               5     16
fedavg 0                    5             4               7     16
fedem  0                    3             5               7     15
pfedme 1                    7             2               3     12
fedavg 1                    3             2               6     11
pfedme 0                    5             5               1     11
fedbn  1                    1             3               3      7
fedem  1                    3             1               2      6
fedbn  0                    0             2               0      2
ditto  0                    1             0               0      1
       1                    0             1               0      1

View processed summaries

In [52]:
# metrics summary
sort_by = ['value', 'rank_one_ind', 'rank_two_ind', 'rank_three_ind']
all_metric_summary.sort_values(by=sort_by, ascending=False)

,rank_one_ind,rank_two_ind,rank_three_ind,value
metric,,,,
Results_unseen/test_acc,4,1,1,6
Results_weighted_unseen/test_acc,4,1,1,6
Results/test_acc_bottom_decile,2,3,1,6
Results_unseen/test_acc_bottom_decile,2,1,3,6
Results/test_acc_std,4,0,1,5
Results/test_acc,1,2,1,4
Results_unseen/test_acc_std,1,1,1,3
Results_weighted/test_acc,0,2,1,3


In [53]:
"""
id_columns = ['method', 'finetune']
show_columns = [name for name in temp.columns if re.search('Results', name)]
for dataset in df.dataset.unique():
    print(dataset)
    display(temp.loc[df.dataset == dataset][id_columns + show_columns].sort_values(by=id_columns))
"""

"\nid_columns = ['method', 'finetune']\nshow_columns = [name for name in temp.columns if re.search('Results', name)]\nfor dataset in df.dataset.unique():\n    print(dataset)\n    display(temp.loc[df.dataset == dataset][id_columns + show_columns].sort_values(by=id_columns))\n"

Now seperately, based on finetuning status, repeat the above summary

In [54]:
(yes_run_summary, yes_metric_summary, _) = process_run_metrics(ft_yes)
print('ft_yes runs:', yes_run_summary.shape)
print('ft_yes metrics:', yes_metric_summary.shape)
(no_run_summary, no_metric_summary, _) = process_run_metrics(ft_no)
print('ft_no runs:', all_run_summary.shape)
print('ft_no metrics:', all_metric_summary.shape)

ft_yes runs: (6, 4)
ft_yes metrics: (8, 4)
ft_no runs: (12, 4)
ft_no metrics: (8, 4)


In [55]:
# run summary
yes_run_summary.sort_values(by='value', ascending=False)

,,rank_one_ind,rank_two_ind,rank_three_ind,value
method,finetune,,,,
exact,1,19,12,4,35
fedavg,1,14,8,5,27
pfedme,1,12,1,12,25
fedem,1,3,6,7,16
fedbn,1,1,6,1,8
ditto,1,0,1,2,3


In [56]:
# metrics summary
yes_metric_summary.sort_values(by=sort_by, ascending=False)

,rank_one_ind,rank_two_ind,rank_three_ind,value
metric,,,,
Results_weighted_unseen/test_acc,5,0,0,5
Results/test_acc_std,3,1,1,5
Results/test_acc_bottom_decile,2,3,0,5
Results_unseen/test_acc,4,0,0,4
Results_unseen/test_acc_bottom_decile,3,0,1,4
Results_unseen/test_acc_std,1,3,0,4
Results/test_acc,1,2,1,4
Results_weighted/test_acc,0,3,1,4


In [57]:
# run summary
no_run_summary.sort_values(by='value', ascending=False)

,,rank_one_ind,rank_two_ind,rank_three_ind,value
method,finetune,,,,
exact,0,19,8,6,33
fedem,0,5,10,18,33
fedavg,0,9,7,9,25
pfedme,0,9,5,2,16
ditto,0,1,0,3,4
fedbn,0,3,0,1,4


In [58]:
# metrics summary
no_metric_summary.sort_values(by=sort_by, ascending=False)

,rank_one_ind,rank_two_ind,rank_three_ind,value
metric,,,,
Results_unseen/test_acc,4,1,0,5
Results_weighted_unseen/test_acc,4,1,0,5
Results/test_acc_std,2,1,2,5
Results/test_acc,2,1,1,4
Results_weighted/test_acc,2,1,1,4
Results_unseen/test_acc_bottom_decile,2,0,2,4
Results/test_acc_bottom_decile,2,1,0,3
Results_unseen/test_acc_std,1,2,0,3
